# Prometeia Financial Benchmark - Benchmarking Language Models in the Financial Domain

## Preliminaries

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.5 MB/s eta 0:00:00


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from google.colab import userdata
from huggingface_hub import login, whoami

import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import copy
import hashlib
from typing import List, Dict, Any, Union
import re

### Huggingface login

In [ ]:
#!hf auth login

login(token=userdata.get('HF_TOKEN'))
whoami()

{'type': 'user',
 'id': '68f8f03ed0d7ccabe590dc77',
 'name': 'carolinaabonafe',
 'fullname': 'Carolina Bonafè',
 'isPro': False,
 'avatarUrl': '/avatars/5ef7b91957cc286f0bf714af57aea49d.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'token_BD',
   'role': 'fineGrained',
   'createdAt': '2025-10-22T14:59:22.006Z',
   'fineGrained': {'canReadGatedRepos': False,
    'global': [],
    'scoped': [{'entity': {'_id': '68f8f03ed0d7ccabe590dc77',
       'type': 'user',
       'name': 'carolinaabonafe'},
      'permissions': []}]}}}}

### Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
MODELS_CACHE= "/content/drive/MyDrive/University/BigData/Project/models_cache"
os.makedirs(MODELS_CACHE, exist_ok=True)
os.environ["HF_HOME"] = MODELS_CACHE
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"



##

## Loading data

In [ ]:
PATH = "/content/drive/MyDrive/University/BigData/Project/data/subtask_1"
SAMPLE_DATA_PATH = PATH + "/sample_data_ITA.tsv"
TEST_DATA_PATH = PATH + "/test_set_unlabelled_IT.tsv"
VALIDATION_DATA_PATH = PATH + "/validation_set_IT.tsv"

In [ ]:
sample_data = pd.read_csv (SAMPLE_DATA_PATH, sep = '\t')
test_data = pd.read_csv (TEST_DATA_PATH, sep = '\t')
validation_data = pd.read_csv (VALIDATION_DATA_PATH, sep = '\t')


## Data Processing

In [ ]:
print(sample_data.shape)
sample_data.head()

(10, 9)


,question,correct_answer,choiceA,choiceB,choiceC,choiceD,choiceE,custom_id,difficulty_level
0,1. L'azienda tradizionale multidomestica ha un...,D,"2, 3, 5","2, 3, 4, 5","1, 2, 3, 4, 5","1, 3, 4",Nessuna delle precedenti,BOOKS__1326,hard
1,1. Il concetto di vantaggio comparato si appli...,D,4,"2, 3, 4, 5",5,"2, 4, 5",Nessuna delle precedenti,BOOKS__1330,hard
2,1. Un'impresa può avere un utile contabile pos...,B,"2, 4","1, 3, 4",Tutte le scelte sono corrette,"1, 2, 3, 4",Nessuna delle precedenti,BOOKS__1411,medium
3,1. Il baby boom si riferisce alla diminuzione ...,C,"1, 2, 3","1, 3",2,Tutte le scelte sono corrette,Nessuna delle precedenti,BOOKS__1686,easy
4,Qual è la percentuale di passività correnti ri...,A,24.08%,25.00%,23.94%,22.05%,Nessuna delle precedenti,FINANCIALS__4358,hard


In [ ]:
print(validation_data.shape)
validation_data.head()

(500, 11)


,custom_id,category,question,choiceA,choiceB,choiceC,choiceD,choiceE,correct_answer,difficulty_level,language
0,BOOKS__1646,BOOKS,1. Il committente massimizza i propri profitti...,1,"1, 2",Tutte le risposte sono corrette.,3,Nessuna delle precedenti,B,medium,IT
1,PAPER__3536,PAPER,1. Il sistema bancario ombra è un tipo di sist...,"1, 4","3, 4",1,"1, 2, 3",Nessuna delle precedenti,C,hard,IT
2,PAPER__3297,PAPER,1. Esiste un equilibrio competitivo se ogni in...,Tutte le risposte sono corrette.,"2, 3",2,"1, 2, 3, 4",Nessuna delle precedenti,C,medium,IT
3,BOOKS__5422,BOOKS,Cosa descrive meglio la caratteristica princip...,A. Le imprese fissano le quantità piuttosto ch...,B. Le aziende si fondono in una grande azienda...,C. Le aziende vendono prodotti identici e comp...,D. Le aziende colludono illegalmente per fissa...,Nessuna delle precedenti,C,medium,IT
4,PAPER__2502,PAPER,1. La misura ∆CoVaR rileva la variazione del v...,4,"2, 5","3, 5","1, 2, 4",Nessuna delle precedenti,D,hard,IT


In [ ]:
y_true_val = validation_data['correct_answer'].tolist()

In [ ]:
print(test_data.shape)
test_data.head()

(1001, 9)


,custom_id,category,question,choiceA,choiceB,choiceC,choiceD,choiceE,language
0,BOOKS__101,BOOKS,Che cos'è l'equivalenza ricardiana nella teori...,L'idea che un aumento della spesa pubblica por...,Il concetto secondo cui i consumatori risparmi...,L'ipotesi che i deficit pubblici portino sempr...,La teoria secondo cui il debito pubblico non i...,Nessuna delle precedenti,IT
1,BOOKS__112,BOOKS,Quale dei seguenti paesi NON è stato citato co...,Taiwan,Giappone,Singapore,Corea del Sud,Nessuna delle precedenti,IT
2,BOOKS__1148,BOOKS,1. Quando l'economia supera il suo prodotto po...,"1, 2, 3","2, 3",3,Tutte le risposte sono corrette.,Nessuna delle precedenti,IT
3,BOOKS__1167,BOOKS,1. La crescita economica nei paesi a basso red...,"1, 2, 3, 4, 5","1, 3, 5","1, 3, 4, 5","1, 5",Nessuna delle precedenti,IT
4,BOOKS__1171,BOOKS,1. La crescita economica dipende esclusivament...,"1, 2","1, 3","1, 3, 4",Tutte le risposte sono corrette.,Nessuna delle precedenti,IT


## Model loading


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
def model_setup(model, bnb_config):
  tokenizer = AutoTokenizer.from_pretrained(model)
  tokenizer.pad_token = tokenizer.eos_token
  #tokenizer.pad_token_id = tokenizer.eos_token_id
  #tokenizer.padding_side = "left"

  terminators= [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  model_mistral = AutoModelForCausalLM.from_pretrained(
      model,
      return_dict=True,
      quantization_config=bnb_config,
      device_map="auto",
      cache_dir= MODELS_CACHE,
      trust_remote_code=True
  )

  return model, tokenizer, terminators



In [ ]:
def is_dir_non_empty(path):
    return os.path.isdir(path) and len(os.listdir(path)) > 0

### Mistral

In [ ]:
model_card_mistral = "mistralai/Mistral-7B-Instruct-v0.3"

load_directory_mistral = "/content/drive/MyDrive/A2/Mistral_Quantized_4bit"
if is_dir_non_empty(load_directory_mistral):
  model_mistral, tokenizer_mistral, terminators_mistral = model_setup(load_directory_mistral, bnb_config)
else:
  model_mistral, tokenizer_mistral, terminators_mistral = model_setup(model_card_mistral, bnb_config)
  print("Quantized model successfully reloaded.")


print(model_mistral)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

In [ ]:
"""# Define the save path on Drive
save_directory = "/content/drive/MyDrive/AssignmentsNLP/Assignment2/Mistral_Quantized_4bit"

# Create the directory if it doesn't exist
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save the Tokenizer
tokenizer_mistral.save_pretrained(save_directory)
print(f"Tokenizer salvato in: {save_directory}")

# Save Model
# Note: When saving a quantized model with BitsAndBytes,
# PyTorch only saves the base state, but you must also save the configuration
# and adapter weights (if any).
model_mistral.save_pretrained(save_directory)
print(f"Model saved in: {save_directory}")

# SAVING TERMINATORS AND CONFIGURATIONS (optional but recommended)
# You can save simple lists like terminators_mistral to a JSON or pickle file.
import json
config_path = os.path.join(save_directory, "config_params.json")
with open(config_path, 'w') as f:
    # We only save the ID values
    json.dump({"terminators": terminators_mistral}, f)
print(f"Configuration saved in: {config_path}")

In [ ]:
model_card_mistral = "mistralai/Mistral-7B-Instruct-v0.3" # Model card from hugging face
tokenizer_mistral = AutoTokenizer.from_pretrained(model_card_mistral)
tokenizer_mistral.pad_token = tokenizer_mistral.eos_token # set the padding tokens

# set the terminators
terminators_mistral = [
    tokenizer_mistral.eos_token_id,
    tokenizer_mistral.convert_tokens_to_ids("<|eot_id|>")
]

model_mistral = AutoModelForCausalLM.from_pretrained(
    model_card_mistral,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
load_directory = "/content/drive/MyDrive/A2/Mistral_Quantized_4bit"

# 1. Load the Tokenizer
tokenizer_mistral = AutoTokenizer.from_pretrained(load_directory)
print("Tokenizer ricaricato con successo.")

tokenizer_mistral.pad_token = tokenizer_mistral.eos_token  # set the padding tokens

# define terminators
terminators_mistral = [
    tokenizer_mistral.eos_token_id,
    tokenizer_mistral.convert_tokens_to_ids("<|eot_id|>")
]

# 2. Load the quantized model (apply bnb_config)
model_mistral = AutoModelForCausalLM.from_pretrained(
    load_directory,
    quantization_config=bnb_config,
    device_map="auto"
)
print("Modello quantizzato ricaricato con successo.")

Tokenizer ricaricato con successo.


/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Modello quantizzato ricaricato con successo.


In [ ]:
print(model_mistral)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

### Llama

In [ ]:
model_card_llama = "meta-llama/Llama-3.1-8B-Instruct"

load_directory_llama = "/content/drive/MyDrive/A2/Llama_Quantized_4bit"
if is_dir_non_empty(load_directory_llama):
  model_llama, tokenizer_llama, terminators_llama = model_setup(load_directory_llama, bnb_config)
else:
  model_llama, tokenizer_llama, terminators_llama = model_setup(model_card_llama, bnb_config)

print(model_llama)

In [ ]:
model_card_llama = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer_llama = AutoTokenizer.from_pretrained(model_card_llama)
tokenizer_llama.pad_token = tokenizer_llama.eos_token
tokenizer_llama.padding_side = "left"  # So that the last token of each sequence in the batch is always a "true" token, and the model can start generating text correctly from that point.

terminators_llama = [
    tokenizer_llama.eos_token_id,
    tokenizer_llama.convert_tokens_to_ids("<|eot_id|>")
]

model_llama = AutoModelForCausalLM.from_pretrained(
    model_card_llama,
    quantization_config=bnb_config,
    device_map="auto"
)

In [ ]:
load_directory = "/content/drive/MyDrive/A2/Llama_Quantized_4bit"

# 1. Load the Tokenizer
tokenizer_llama = AutoTokenizer.from_pretrained(load_directory)
print("Tokenizer ricaricato con successo.")

tokenizer_llama.pad_token = tokenizer_llama.eos_token  # set the padding tokens
tokenizer_llama.padding_side = "left"

# define terminators
terminators_llama = [
    tokenizer_llama.eos_token_id,
    tokenizer_llama.convert_tokens_to_ids("<|eot_id|>")
]

# 2. Load the quantized model (apply bnb_config)
model_llama = AutoModelForCausalLM.from_pretrained(
    load_directory,
    quantization_config=bnb_config,
    device_map="auto"
)
print("Modello quantizzato ricaricato con successo.")

## Zero-Shot Inference

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'Sei un esperto di finanza e macroeconomia.Rispondi solo con la lettera della risposta corretta.'
    },
    {
        'role': 'user',
        'content': """Il tuo compito è di rispondere alla seguente
         domanda a scelta multipla basandoti esclusivamente sulle tue
         conoscenze o sul contesto fornito.

         Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.


         Domanda: {question}

          A) {choiceA}
          B) {choiceB}
          C) {choiceC}
          D) {choiceD}
          E) {choiceE}

         Risposta:"
        """
    }
]

In [ ]:
def prepare_prompts(df, prompt_template, tokenizer):
    """
      This function format input text samples into instructions prompts.

      Inputs:
        texts: input texts to classify via prompting
        prompt_template: the prompt template provided in this assignment
        tokenizer: the transformers Tokenizer object instance associated
        with the chosen model card

      Outputs:
        input texts to classify in the form of instruction prompts
    """
    formatted_prompts = []

    generation_kwargs = {
        'tokenize': False,
        'add_generation_prompt': True
    }

    user_content_template = prompt_template[1]['content']

    # Iteriamo sulle righe del DataFrame
    for _, row in df.iterrows():
        # Formattiamo il contenuto dell'utente con i campi del CSV
        user_content = user_content_template.format(
            question=row['question'],
            choiceA=row['choiceA'],
            choiceB=row['choiceB'],
            choiceC=row['choiceC'],
            choiceD=row['choiceD'],
            choiceE=row['choiceE']
        )

        current_prompt = [
            prompt_template[0],
            {'role': 'user', 'content': user_content}
        ]

        try:
            formatted_string = tokenizer.apply_chat_template(
                current_prompt,
                **generation_kwargs
            )
            formatted_prompts.append(formatted_string)
        except Exception as e:
            print(f"Error formatting row {row.get('custom_id', 'unknown')}: {e}")
            formatted_prompts.append(None)

    return formatted_prompts

In [ ]:
def generate_responses(model, prompt_examples, tokenizer, batch_size=8):
    """
      This function implements the inference loop for a LLM model.
      Given a set of examples, the model is tasked to generate
      a response.

      Inputs:
        model: LLM model instance for prompting
        prompt_examples: pre-processed text samples
        tokenizer: the corresponding Tokenizer instance (required for tokenization)
        batch_size: how many samples to process at once

      Outputs:
        generated responses
    """

    generated_texts = []
    model.eval()

    for i in range(0, len(prompt_examples), batch_size):
        batch = [p for p in prompt_examples[i:i + batch_size] if p is not None]
        if not batch: continue

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048
        ).to(model.device)

        generation_config = {
            "max_new_tokens": 20,
            "do_sample": False,
            "temperature": 0.0,           # Per massima precisione
            "pad_token_id": tokenizer.pad_token_id,
            "eos_token_id": tokenizer.eos_token_id,
        }

        with torch.no_grad():
            outputs = model.generate(**inputs, **generation_config)

        for i, output_sequence in enumerate(outputs):
            start_index = inputs['input_ids'][i].shape[0]
            generated_sequence = output_sequence[start_index:]

            decoded_text = tokenizer.decode(
                generated_sequence,
                skip_special_tokens=True
            ).strip()
            generated_texts.append(decoded_text)

    return generated_texts

In [ ]:
def process_response(response):
    """
      This function takes a textual response generated by the LLM
      and processes it to map the response to a binary label.

      Inputs:
        response: generated response from LLM

      Outputs:
        parsed classification response.
    """

    # Pulizia base
    cleaned = response.strip().upper()

    # Cerchiamo la prima occorrenza di A, B, C, D o E all'inizio del testo
    # o seguita da una parentesi/punto
    match = re.search(r'\b([A-E])\b', cleaned)

    if match:
        return match.group(1)

    # Se il modello scrive "La risposta corretta è la B", cerchiamo la lettera
    if len(cleaned) > 0 and cleaned[0] in "ABCDE":
        return cleaned[0]

    return "N/A" # Valore per risposte non parseabili

In [ ]:
def compute_metrics(y_true, y_pred, label_name="Model"):
    """
    Calcola e stampa Accuracy, F1-Score, Fail Ratio e Confusion Matrix.
    """
    # 1. Identifica i fallimenti (N/A)
    # Consideriamo "N/A" o qualsiasi cosa non sia A, B, C, D, E come un fallimento
    valid_labels = ['A', 'B', 'C', 'D', 'E']

    # Pulizia: convertiamo tutto in stringa e upper
    y_true = [str(x).upper().strip() for x in y_true]
    y_pred = [str(x).upper().strip() for x in y_pred]

    # Calcolo Fail Ratio (quante volte il modello non ha prodotto una lettera valida)
    fails = sum(1 for p in y_pred if p not in valid_labels)
    fail_ratio = fails / len(y_pred)

    # 2. Calcolo Accuratezza (N/A conta come errore)
    accuracy = accuracy_score(y_true, y_pred)

    # 3. Calcolo F1-Score (Macro)
    # Usiamo 'macro' per dare lo stesso peso a ogni classe (A, B, C, D, E)
    # indipendentemente dalla loro frequenza nel dataset
    f1 = f1_score(y_true, y_pred, average='macro', labels=valid_labels)

    # 4. Matrice di Confusione
    # Includiamo 'N/A' nelle predizioni per vedere dove il modello si blocca
    all_labels = valid_labels + ['N/A']
    # Mappiamo le predizioni non valide a 'N/A' per la matrice
    y_pred_mapped = [p if p in valid_labels else 'N/A' for p in y_pred]

    cm = confusion_matrix(y_true, y_pred_mapped, labels=all_labels)

    # --- Stampa dei risultati ---
    print(f"\n" + "="*30)
    print(f"📊 METRICHE PER: {label_name}")
    print(f"="*30)
    print(f"Accuracy:   {accuracy:.2%}")
    print(f"F1-Score:   {f1:.4f} (Macro)")
    print(f"Fail Ratio: {fail_ratio:.2%} ({fails}/{len(y_pred)} risposte non valide)")
    print("-" * 30)

    # Visualizzazione Matrice di Confusione
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=all_labels, yticklabels=valid_labels)
    plt.title(f"Confusion Matrix - {label_name}")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return {
        "accuracy": accuracy,
        "f1": f1,
        "fail_ratio": fail_ratio,
        "confusion_matrix": cm
    }

### Mistral

In [ ]:
tokenizer=tokenizer_mistral
model = model_mistral

In [ ]:
prompts = prepare_prompts(validation_data, prompt, tokenizer)

In [ ]:
raw_responses = generate_responses(model, prompts, tokenizer)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)


Output of generate_responses ('Raw' Model Responses)

['La risposta corretta è D) 3. Se un agente è neutrale al', 'La risposta corretta è B) 3, 4.\n\n1. Il', 'La risposta corretta è D) 1, 2, 3, 4', 'C) C. Le aziende vendono prodotti identici e competano fissando', 'La risposta corretta è D) 1, 2, 4.', 'La risposta corretta è D) 50.576. Il reddito', 'D) 2, 3, 4', 'C) Maggiore esposizione ai rischi di coda che potrebbero', 'B) Il grado di allineamento dei movimenti congiunti del capitale', 'La risposta è D) 11,46%. Il magazzino è indic', 'La risposta corretta è C) 2, 4.\n\n2. L', 'La risposta corretta è A) 1, 2, 4, 5', 'C) Un rischio più elevato è associato a rendimenti attesi inferiori', '2) Le politiche governative durante una recessione possono includere strument', 'C) Rischio di carenza di capitale (CSR)', 'C) Le loro prestazioni sono generalmente inferiori a causa degli elevati costi di trans', 'A) A. Un surplus di beni si verifica quando i produttori offron', 'La risposta è B) 480

In [ ]:
predictions = [process_response(r) for r in raw_responses]
validation_data['prediction'] = predictions

In [ ]:
print("\nOutput of process_response (Parsed Answers)\n")
print(predictions)


Output of process_response (Parsed Answers)

['D', 'B', 'D', 'C', 'D', 'D', 'D', 'C', 'B', 'D', 'C', 'A', 'C', 'N/A', 'C', 'C', 'A', 'B', 'B', 'C', 'C', 'E', 'B', 'E', 'N/A', 'B', 'C', 'A', 'B', 'C', 'E', 'C', 'B', 'N/A', 'D', 'C', 'C', 'N/A', 'B', 'A', 'C', 'C', 'B', 'C', 'D', 'B', 'C', 'A', 'B', 'C', 'D', 'B', 'B', 'B', 'B', 'A', 'D', 'E', 'D', 'N/A', 'B', 'E', 'B', 'A', 'A', 'C', 'C', 'B', 'B', 'B', 'A', 'D', 'C', 'B', 'B', 'E', 'D', 'N/A', 'N/A', 'E', 'C', 'E', 'B', 'A', 'B', 'C', 'D', 'C', 'B', 'B', 'B', 'C', 'B', 'C', 'A', 'N/A', 'N/A', 'B', 'C', 'N/A', 'B', 'D', 'A', 'C', 'B', 'A', 'D', 'C', 'D', 'A', 'B', 'B', 'B', 'C', 'D', 'E', 'B', 'C', 'A', 'C', 'D', 'N/A', 'C', 'B', 'A', 'B', 'A', 'D', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'N/A', 'E', 'D', 'B', 'D', 'A', 'C', 'A', 'B', 'B', 'C', 'B', 'C', 'A', 'D', 'D', 'C', 'D', 'C', 'N/A', 'D', 'B', 'C', 'N/A', 'B', 'A', 'A', 'B', 'E', 'C', 'B', 'A', 'C', 'A', 'B', 'D', 'D', 'C', 'A', 'C', 'C', 'D', 'D', 'D', 'B', 'B', 'B', 'E', 'A', 'E', 

In [ ]:
# Calcola accuratezza semplice
accuracy = (validation_data['prediction'] == validation_data['correct_answer']).mean()
print(f"Accuracy Zero-Shot: {accuracy:.2%}")

Accuracy Zero-Shot: 40.60%


In [ ]:
mistral_metrics = compute_metrics(y_true_val, predictions, 'Mistral')
mistral_metrics

### Llama

In [ ]:
tokenizer=tokenizer_llama
model = model_llama

In [ ]:
prompts = prepare_prompts(validation_data, prompt, tokenizer)

In [ ]:
raw_responses = generate_responses(model, prompts, tokenizer)

In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)

In [ ]:
predictions = [process_response(r) for r in raw_responses]
validation_data['prediction'] = predictions

In [ ]:
print("\nOutput of process_response (Parsed Answers)\n")
print(predictions)

In [ ]:
# Calcola accuratezza semplice
accuracy = (validation_data['prediction'] == validation_data['correct_answer']).mean()
print(f"Accuracy Zero-Shot: {accuracy:.2%}")

In [ ]:
llama_metrics = compute_metrics(y_true_val, predictions, 'Llama')
llama_metrics

### Zero-Shot ma Mistral usato come Scorer

In [ ]:
tokenizer=tokenizer_mistral
model = model_mistral

In [ ]:
def build_scoring_prompt(question, answer):
    return [
        {
            "role": "system",
            "content": "Sei un esperto di finanza e macroeconomia. Rispondi solo con Sì o No."
        },
        {
            "role": "user",
            "content": f"""
Domanda:
{question}

Risposta:
{answer}

Questa risposta è corretta?
"""
        }
    ]


In [ ]:
import torch

def score_yes_probability(model, tokenizer, prompt):
    """
    Ritorna la probabilità che il modello risponda 'Sì'
    """

    formatted_prompt = tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)

    # Logits dell'ULTIMO token (quello da generare)
    logits = outputs.logits[:, -1, :]

    # Token ids per "Sì" e "No"
    yes_token_id = tokenizer.encode("Sì", add_special_tokens=False)[0]
    no_token_id  = tokenizer.encode("No", add_special_tokens=False)[0]

    # Softmax SOLO sui due token
    probs = torch.softmax(
        logits[:, [yes_token_id, no_token_id]],
        dim=-1
    )

    prob_yes = probs[0, 0].item()
    return prob_yes


In [ ]:
def predict_answer(question, choices, model, tokenizer):
    """
    choices = lista di risposte candidate (stringhe)
    """

    scores = []

    for choice in choices:
        prompt = build_scoring_prompt(question, choice)
        score = score_yes_probability(model, tokenizer, prompt)
        scores.append(score)

    best_idx = int(torch.tensor(scores).argmax())
    return best_idx, scores


In [ ]:
predictions = []

for _, row in validation_data.iterrows():
    question = row["question"]
    choices = [
        row["choiceA"],
        row["choiceB"],
        row["choiceC"],
        row["choiceD"],
        row["choiceE"]
    ]

    pred_idx, scores = predict_answer(
        question,
        choices,
        model,
        tokenizer
    )

    predicted_letter = ["A", "B", "C", "D", "E"][pred_idx]
    predictions.append(predicted_letter)

validation_data["prediction"] = predictions


In [ ]:
accuracy = (
    validation_data["prediction"]
    == validation_data["correct_answer"]
).mean()

print(f"Accuracy (Mistral scorer): {accuracy:.2%}")

Accuracy (Mistral scorer): 48.20%


### Case-study: question difficulty

In [ ]:
tokenizer=tokenizer_mistral
model = model_mistral

#### Easy

In [ ]:
easy_data = validation_data[validation_data['difficulty_level']=='easy']
easy_data.head()

In [ ]:
easy_data.shape

In [ ]:
prompts = prepare_prompts(easy_data, prompt, tokenizer)

In [ ]:
raw_responses = generate_responses(model, prompts, tokenizer)

In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)

In [ ]:
predictions = [process_response(r) for r in raw_responses]
easy_data['prediction'] = predictions

In [ ]:
y_true = easy_data['correct_answer'].tolist()
mistral_metrics = compute_metrics(y_true, predictions, 'Mistral')
mistral_metrics

#### Medium

In [ ]:
medium_data = validation_data[validation_data['difficulty_level']=='medium']
medium_data.head()

In [ ]:
medium_data.shape

In [ ]:
prompts = prepare_prompts(medium_data, prompt, tokenizer)

In [ ]:
raw_responses = generate_responses(model, prompts, tokenizer)

In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)

In [ ]:
predictions = [process_response(r) for r in raw_responses]
medium_data['prediction'] = predictions

In [ ]:
y_true = medium_data['correct_answer'].tolist()
mistral_metrics = compute_metrics(y_true, predictions, 'Mistral')
mistral_metrics

#### Hard

In [ ]:
hard_data = validation_data[validation_data['difficulty_level']=='hard']
hard_data.head()

In [ ]:
hard_data.shape

In [ ]:
prompts = prepare_prompts(hard_data, prompt, tokenizer)

In [ ]:
raw_responses = generate_responses(model, prompts, tokenizer)

In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)

In [ ]:
predictions = [process_response(r) for r in raw_responses]
hard_data['prediction'] = predictions

In [ ]:
y_true = hard_data['correct_answer'].tolist()
mistral_metrics = compute_metrics(y_true, predictions, 'Mistral')
mistral_metrics

## Few-Shot Inference

In [ ]:
prompt_template_few_shot = [
    {
        'role': 'system',
        'content': 'Sei un esperto di finanza e macroeconomia. Rispondi solo con la lettera della risposta corretta.'
    },
    {
        'role': 'user',
        'content': """l tuo compito è di rispondere alla seguente
         domanda a scelta multipla basandoti esclusivamente sulle tue
         conoscenze o sul contesto fornito.

         Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.

         Questa è la lista di esempi dalla quale puoi prendere spunto:

{examples}

Ora rispondi alla seguente domanda:
Domanda: {question}

A) {choiceA}
B) {choiceB}
C) {choiceC}
D) {choiceD}
E) {choiceE}

Risposta:"""
    }
]



In [ ]:
def build_few_shot_demonstrations(
    demonstrations: pd.DataFrame,
    num_per_class: int = 1,
    shuffle: bool = True,
    random_state: int = 42
) -> str:
    """
    Seleziona esempi dal sample_set e li formatta come stringa.
    """
    demo_samples = []

    # Per ogni possibile risposta (A-E), prendiamo num_per_class esempi
    for label in ['A', 'B', 'C', 'D', 'E']:
        class_subset = demonstrations[demonstrations['correct_answer'] == label]
        if not class_subset.empty:
            sample = class_subset.sample(min(len(class_subset), num_per_class), random_state=random_state)
            demo_samples.append(sample)

    selected_demos = pd.concat(demo_samples)
    if shuffle:
        selected_demos = selected_demos.sample(frac=1, random_state=random_state)

    demo_string = ""
    for _, row in selected_demos.iterrows():
        demo_string += f"Domanda: {row['question']}\n"
        demo_string += f"A) {row['choiceA']}\nB) {row['choiceB']}\nC) {row['choiceC']}\nD) {row['choiceD']}\nE) {row['choiceE']}\n"
        demo_string += f"Risposta: {row['correct_answer']}\n\n---\n\n"

    return demo_string


In [ ]:
def prepare_prompts_few_shot(
    target_df: pd.DataFrame,  # Cambiato da List[str] a DataFrame
    prompt_template: List[Dict[str, str]],
    tokenizer: Any,
    demonstrations_df: pd.DataFrame,
    num_per_class: int = 1, # Nota: 1 per classe = 5 esempi totali.
    shuffle_examples: bool = True,
) -> List[str]:

    formatted_prompts = []

    # Gestione specifica per modelli "Reasoning" (Qwen/DeepSeek)
    is_qwen_tokenizer = hasattr(tokenizer, 'chat_template') and \
                        tokenizer.chat_template and \
                        'think' in str(tokenizer.chat_template)

    generation_kwargs = {
        'tokenize': False,
        'add_generation_prompt': True
    }
    if is_qwen_tokenizer:
        generation_kwargs['enable_thinking'] = False

    user_content_base = prompt_template[1]['content']

    for _, row in target_df.iterrows():
        # Generiamo un seed basato sulla domanda per avere determinismo
        seed_value = int(hashlib.sha1(row['question'].encode('utf-8')).hexdigest(), 16) % 10**8

        # Costruiamo il blocco esempi
        demonstrations_string = build_few_shot_demonstrations(
            demonstrations=demonstrations_df,
            num_per_class=num_per_class,
            shuffle=shuffle_examples,
            random_state=seed_value
        )

        # Inseriamo gli esempi e poi i dati della riga corrente
        user_content = user_content_base.replace('{examples}', demonstrations_string)
        user_content = user_content.format(
            question=row['question'],
            choiceA=row['choiceA'],
            choiceB=row['choiceB'],
            choiceC=row['choiceC'],
            choiceD=row['choiceD'],
            choiceE=row['choiceE']
        )

        current_prompt = [
            prompt_template[0],
            {'role': 'user', 'content': user_content}
        ]

        try:
            formatted_string = tokenizer.apply_chat_template(
                current_prompt,
                **generation_kwargs
            )
            formatted_prompts.append(formatted_string)
        except Exception as e:
            print(f"Error formatting ID {row.get('custom_id', 'unknown')}: {e}")
            formatted_prompts.append(None)

    return formatted_prompts

In [ ]:
prompts = prepare_prompts_few_shot(
    target_df=validation_data,
    prompt_template=prompt_template_few_shot,
    tokenizer=tokenizer,
    demonstrations_df=sample_data,
    num_per_class=1 # Totale 5 esempi (uno per ogni possibile risposta)
)

In [ ]:
print(prompts[0])

<s>[INST] Sei un esperto di finanza e macroeconomia. Rispondi solo con la lettera della risposta corretta.

l tuo compito è di rispondere alla seguente
         domanda a scelta multipla basandoti esclusivamente sulle tue
         conoscenze o sul contesto fornito.

         Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.

         Questa è la lista di esempi dalla quale puoi prendere spunto:

Domanda: Qual è la percentuale di passività correnti rispetto al totale delle attività al 31 dicembre 2022?

## TABELLA
**STATO PATRIMONIALE CONSOLIDATO AL 31 DICEMBRE 2022**
(Migliaia di euro)

| ATTIVITÀ | Note | 31/12/2022 | 31/12/2021 | PATRIMONIO E PASSIVITÀ | Note | 31/12/2022 | 31/12/2021 |
|--------|-------|------------|------------|-------------------------|-------|------------|------------|
| **ATTIVITÀ NON CORRENTI** | | **PATRIMONIO NETTO** | | | | | | |
| Avviamento | 4 | 792.897 | 725.789 | Capitale sociale | 12.a | 12.356 | 13.070 |
| Altr

In [ ]:
raw_responses = generate_responses(model, prompts, tokenizer, batch_size=4)

In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)


Output of generate_responses ('Raw' Model Responses)

['costi che non sono registrati nel bilancio.\n3. Il valore di merc', 'li investimenti in azioni sono considerati un tipo di investimento di tipo passivo.', 'E) Nessuna delle precedenti', 'del modello organizzativo transnazionale?\nA) Aziende che hanno una', 'D) 1, 2, 4', 'LA\n### **Risultato complessivo attribuibile a:**\n(Importi', 'Il boom economico si riferisce alla crescita economica negli anni dal 1', 'iguarda il credito?\nA) Il credito diventa più accessibile per le', 'B) Il grado di allineamento dei movimenti congiunti del capitale', '6) 1.011 | 1.007 | | Passività', 'B) Tutte le risposte sono corrette.', 'i contanti sono un tipo di sconto che viene applicato quando un cliente paga', 'imento?\nA) Il rischio di un portafoglio è proporzionale', 'B) 1\n\nLa recessione causata dalla pandemia non ha avuto', 'C) Rischio di carenza di capitale (CSR)', 'C) Le loro prestazioni sono generalmente inferiori a causa degli elevati costi 

In [ ]:
predictions = [process_response(r) for r in raw_responses]
validation_data['prediction'] = predictions

In [ ]:
print("\nOutput of process_response (Parsed Answers)\n")
print(predictions)


Output of process_response (Parsed Answers)

['C', 'N/A', 'E', 'A', 'D', 'A', 'N/A', 'A', 'B', 'N/A', 'B', 'N/A', 'A', 'B', 'C', 'C', 'N/A', 'A', 'N/A', 'N/A', 'A', 'E', 'B', 'A', 'B', 'N/A', 'N/A', 'N/A', 'C', 'A', 'A', 'A', 'N/A', 'A', 'N/A', 'C', 'E', 'C', 'A', 'C', 'C', 'N/A', 'C', 'N/A', 'E', 'B', 'A', 'N/A', 'C', 'E', 'N/A', 'A', 'A', 'B', 'A', 'A', 'A', 'N/A', 'N/A', 'N/A', 'N/A', 'E', 'N/A', 'A', 'N/A', 'C', 'C', 'N/A', 'N/A', 'C', 'A', 'D', 'N/A', 'E', 'A', 'D', 'N/A', 'A', 'A', 'E', 'C', 'E', 'N/A', 'N/A', 'B', 'A', 'A', 'C', 'A', 'N/A', 'B', 'N/A', 'D', 'C', 'N/A', 'N/A', 'N/A', 'N/A', 'C', 'N/A', 'C', 'N/A', 'N/A', 'N/A', 'E', 'A', 'N/A', 'C', 'D', 'E', 'A', 'N/A', 'C', 'N/A', 'A', 'D', 'E', 'N/A', 'C', 'N/A', 'A', 'N/A', 'C', 'N/A', 'N/A', 'N/A', 'B', 'C', 'A', 'D', 'N/A', 'C', 'N/A', 'C', 'A', 'N/A', 'N/A', 'D', 'E', 'N/A', 'N/A', 'C', 'A', 'N/A', 'C', 'N/A', 'B', 'A', 'C', 'N/A', 'D', 'B', 'N/A', 'C', 'N/A', 'A', 'N/A', 'N/A', 'A', 'N/A', 'N/A', 'D', 'A', 'N/A', 'C', 'N

In [ ]:
# Calcola accuratezza semplice
accuracy = (validation_data['prediction'] == validation_data['correct_answer']).mean()
print(f"Accuracy Zero-Shot: {accuracy:.2%}")

Accuracy Zero-Shot: 22.60%


## Instruction-tuned prompting

In [ ]:
PROMPTS = {
            "free": [
                      {'role': 'system', 'content': 'Rispondi alla domanda a scelta multipla. Rispondi solo con la lettera corretta.'},
                      {'role': 'user', 'content': """Domanda a scelta multipla: {question}

                      A) {choiceA}
                      B) {choiceB}
                      C) {choiceC}
                      D) {choiceD}
                      E) {choiceE}

                      Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.

                      Risposta:"""}
                  ],

            "verify": [
                        {
                            'role': 'system',
                            'content': 'Sei un esperto di finanza e macroeconomia. Controlla attentamente le opzioni due volte prima di rispondere. Rispondi solo con la lettera.'
                        },
                        {
                            'role': 'user',
                            'content': """Leggi la domanda a scelta multipla e verifica due volte quale opzione è corretta.
                            Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.

                        Domanda: {question}

                        A) {choiceA}
                        B) {choiceB}
                        C) {choiceC}
                        D) {choiceD}
                        E) {choiceE}

                        Risposta:"""
                        }
                      ],

            "confidence": [
                            {
                                'role': 'system',
                                'content': 'Sei un esperto di finanza e macroeconomia. Analizza i problemi in modo analitico e rigoroso e rispondi con la lettera corretta.'
                            },
                            {
                                'role': 'user',
                                'content': """Il tuo compito è di rispondere alla seguente domanda a scelta multipla basandoti esclusivamente sulle tue conoscenze o sul contesto fornito.

                                Domanda: {question}

                                A) {choiceA}
                                B) {choiceB}
                                C) {choiceC}
                                D) {choiceD}
                                E) {choiceE}

                                Istruzioni:
                                1. Valuta attentamente ogni opzione.
                                2. Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.
                                3. Indica quanto sei sicuro della tua risposta su una scala da 1 a 5 (5 = molto sicuro). Fornire solo il numero senza alcuna spiegazione.

                                CRITICAL: Devi rispondere **solo** nel seguente formato. Non aggiungere preamboli o testo extra:

                                Risposta: [A or B or C or D or E]
                                Confidence: [1-5]
                                """
                                    }
                                ],

            "contrastive":  [
                            {
                                'role': 'system',
                                'content': 'Sei un esperto di finanza e macroeconomia. Analizza i problemi in modo analitico e rigoroso.'
                            },
                            {
                                'role': 'user',
                                'content': """Il tuo compito è di rispondere alla seguente domanda a scelta multipla basandoti esclusivamente sulle tue conoscenze o sul contesto fornito.

                              Domanda: {question}

                              A) {choiceA}
                              B) {choiceB}
                              C) {choiceC}
                              D) {choiceD}
                              E) {choiceE}

                              Istruzioni:
                              1. Analizza tutte le opzioni.
                              2. Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.
                              3. Fornisci una breve spiegazione del perché le altre opzioni sono errate.

                              CRITICAL: Devi rispondere **solo** nel seguente formato. Non aggiungere preamboli o testo extra:

                              Risposta: [A or B or C or D or E]
                              Contrast_analysis: [breve spiegazione perché le altre opzioni sono errate]
                              """
                                  }
                              ],

            "high_structure": [
                                    {'role': 'system', 'content': 'Sei un docente esperto di economia aziendale. Analizza ogni opzione e valuta se è vera o falsa, poi fornisci solo la lettera corretta.'},
                                    {'role': 'user', 'content': """ Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.

                                    Domanda a scelta multipla: {question}

                                Opzioni:
                                A) {choiceA}
                                B) {choiceB}
                                C) {choiceC}
                                D) {choiceD}
                                E) {choiceE}


                                Risposta:"""}
                              ]

}



In [ ]:

def _empty_output(prompt_type):
    if prompt_type == "confidence":
        return {"RISPOSTA": "N/A", "CONFIDENCE": "N/A"}
    if prompt_type == "contrastive":
        return {"RISPOSTA": "N/A", "CONTRAST_ANALYSIS": "N/A"}
    return "N/A"


VALID_LETTERS = {"A", "B", "C", "D", "E"}


def process_response_prompting_tuning(response, prompt_type="high_structure", strict=False):
    """
    Estrae e valida i campi principali dalle risposte generate.

    Inputs:
        response (str): testo generato dal modello
        prompt_type (str): cot | confidence | contrastive | high_structure | verify
        strict (bool): se True accetta SOLO il formato esplicito richiesto

    Outputs:
        - cot / verify / high_structure -> str (A–E o "N/A")
        - confidence -> dict {RISPOSTA, CONFIDENCE}
        - contrastive -> dict {RISPOSTA, CONTRAST_ANALYSIS}
    """

    if not response or not response.strip():
        return _empty_output(prompt_type)

    response = response.strip()

    match_explicit = re.search(
        r"Risposta\s*:\s*([A-E])\b", response, re.IGNORECASE
    )

    answer_letter = None
    if match_explicit:
        answer_letter = match_explicit.group(1).upper()


    if not answer_letter and not strict:
        # Pattern tipo: "la risposta corretta è C", "quindi D è giusta"
        fallback_matches = re.findall(
            r"\b([A-E])\b(?=.{0,30}(corretta|giusta|vera|risposta))",
            response.lower()
        )
        if fallback_matches:
            answer_letter = fallback_matches[-1][0].upper()

    # Ultimo fallback: ultima lettera A–E nel testo
    if not answer_letter and not strict:
        all_letters = re.findall(r"\b([A-E])\b", response.upper())
        if all_letters:
            answer_letter = all_letters[-1]


    if answer_letter not in VALID_LETTERS:
        answer_letter = "N/A"


    if prompt_type == "confidence":
        match_conf = re.search(r"Confidence\s*:\s*([1-5])\b", response)
        confidence = match_conf.group(1) if match_conf else "N/A"

        return {
            "RISPOSTA": answer_letter,
            "CONFIDENCE": confidence
        }


    if prompt_type == "contrastive":
        match_contrast = re.search(
            r"Contrast_analysis\s*:\s*(.*)",
            response,
            re.IGNORECASE | re.DOTALL
        )
        contrast = match_contrast.group(1).strip() if match_contrast else "N/A"

        return {
            "RISPOSTA": answer_letter,
            "CONTRAST_ANALYSIS": contrast
        }

    return answer_letter




In [ ]:
#da rivedere
def prompting_tuning(model, validation data, tokenizer, ):

all_results = {}

for prompt_name, prompt_template in PROMPTS.items():
    print(f"Running prompt: {prompt_name}")

    prompts = prepare_prompts(validation_data, prompt_template, tokenizer)

    raw_responses = generate_responses(
        model,
        prompts,
        tokenizer,
        batch_size=4
    )

    # 3. Parsing
    predictions = [
        process_response_prompting_tuning(r, prompt_type=prompt_name)
        for r in raw_responses
    ]

    all_results[prompt_name] = predictions

return all_results

## Multiple-answer generation with selection of a single best response

Per "multiple-answer generation with selection of a single best response" invece pensavamo di cercare un modello giudice che andasse a selezionare la risposta più corretta.

## Chain-of-thought (CoT) prompting

In [ ]:
prompt_template_cot = [
    {
        'role': 'system',
        'content': 'Sei un esperto di finanza e macroeconomia. Analizza i problemi in modo analitico e rigoroso.'
    },
    {
        'role': 'user',
        'content': """Il tuo compito è di rispondere alla seguente
         domanda a scelta multipla basandoti esclusivamente sulle tue
         conoscenze o sul contesto fornito.

        Domanda: {question}

        A) {choiceA}
        B) {choiceB}
        C) {choiceC}
        D) {choiceD}
        E) {choiceE}

        Istruzioni:
        1. Identifica l'argomento principale (es. contabilità, politica fiscale, macroeconomia).
        2. Valuta brevemente ogni opzione (A, B, C, D, E).
        3. Rispondi fornendo solo la lettera (A, B, C, D, o E) corrispondente alla risposta corretta.

        IMPORTANT: Even if multiple categories apply, select ONLY the ONE most fitting category.

        CRITICAL: You must respond with ONLY the following format. Do not add any preamble, explanation, or extra text:

        REASONING: [Your step-by-step analysis here]
        RISPOSTA: [A or B or C or D or E]
        """
    }
]

In [ ]:
# TODO: da unire con generate_response
def generate_responses_cot(model, prompt_examples, tokenizer, batch_size=4):
    generated_texts = []
    model.eval()

    for i in range(0, len(prompt_examples), batch_size):
        batch = [p for p in prompt_examples[i:i + batch_size] if p is not None]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=2048).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=450,  # Aumentato per permettere il ragionamento
                do_sample=False,     # Manteniamo deterministico
                temperature=0,
                pad_token_id=tokenizer.pad_token_id
            )

        for j, output_sequence in enumerate(outputs):
            start_index = inputs['input_ids'][j].shape[0]
            decoded_text = tokenizer.decode(output_sequence[start_index:], skip_special_tokens=True).strip()
            generated_texts.append(decoded_text)

    return generated_texts

In [ ]:
prompts = prepare_prompts(validation_data, prompt_template_cot, tokenizer)

In [ ]:
raw_responses = generate_responses_cot(model, prompts, tokenizer)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
print("\nOutput of generate_responses ('Raw' Model Responses)\n")
print(raw_responses)

In [ ]:
def process_response_cot(response):
    if not response: return "N/A"

    # 1. Cerca il trigger specifico richiesto
    match_finale = re.search(r"RISPOSTA:\s*([A-E])", response, re.IGNORECASE)
    if match_finale:
        return match_finale.group(1).upper()

    # 2. Se non c'è il trigger, cerca l'ultima lettera associata a una scelta corretta/vera
    # Spesso il modello scrive: "Quindi la C è corretta"
    match_inferenza = re.findall(r"([A-E])\b(?=.{0,20}(?:corretta|vera|giusta|risposta))", response.lower())
    if match_inferenza:
        return match_inferenza[-1].upper()

    # 3. Fallback: l'ultima lettera A-E trovata nel testo (spesso la conclusione)
    # Escludiamo le lettere che sembrano far parte di tabelle o elenchi iniziali
    clean_text = re.sub(r'^[1-5]\.\s', '', response) # Rimuove numerazioni iniziali
    all_letters = re.findall(r"\b([A-E])\b", clean_text.upper())
    if all_letters:
        return all_letters[-1]

    return "N/A"

In [ ]:
predictions = [process_response_cot(r) for r in raw_responses]
validation_data['prediction_cot'] = predictions

In [ ]:
print("\nOutput of process_response (Parsed Answers)\n")
print(predictions)

In [ ]:
# Calcola accuratezza semplice
accuracy = (validation_data['prediction_cot'] == validation_data['correct_answer']).mean()
print(f"Accuracy Zero-Shot: {accuracy:.2%}")

## Generate and Read Pipeline

In [ ]:
def get_logit_scores(model, tokenizer, final_prompt):
    """
    Invece di generare testo, calcola la probabilità matematica delle lettere A-E.
    """
    choices = ['A', 'B', 'C', 'D', 'E']
    inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        # Prendiamo i logits dell'ultimo token predetto
        last_token_logits = outputs.logits[0, -1, :]

    # Mappiamo i token ID delle lettere (aggiungendo uno spazio davanti spesso aiuta)
    # Molti modelli chat usano formati diversi, testiamo la lettera secca
    choice_ids = [tokenizer.encode(c, add_special_tokens=False)[-1] for c in choices]

    # Estraiamo e normalizziamo con Softmax
    target_logits = last_token_logits[choice_ids]
    probs = F.softmax(target_logits, dim=-1)

    # Risultato
    prediction = choices[torch.argmax(probs).item()]
    confidences = {choices[i]: probs[i].item() for i in range(len(choices))}

    return prediction, confidences

In [ ]:
import torch.nn.functional as F
from tqdm import tqdm

In [ ]:
def run_generate_and_read(df, model, tokenizer):
    results = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Pipeline G&R"):
        # --- STEP 1: GENERATE (Reasoning) ---
        # Chiediamo al modello di estrarre solo i fatti numerici o logici
        gen_prompt = f"""[INST] Sei un analista finanziario. Analizza i dati della seguente domanda e della tabella fornita.
Identifica i numeri chiave e la logica necessaria per rispondere, senza dare ancora la risposta finale.

Domanda: {row['question']}
[/INST]
Ragionamento:"""

        inputs = tokenizer(gen_prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            gen_output = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=False,
                temperature=0.0
            )

        reasoning = tokenizer.decode(gen_output[0][inputs.input_ids.shape[1]:], skip_special_tokens=True).strip()

        # --- STEP 2: READ & SCORE ---
        # Costruiamo il prompt finale che include il ragionamento appena fatto
        final_scorer_prompt = f"""[INST] Domanda: {row['question']}

Opzioni:
A) {row['choiceA']}
B) {row['choiceB']}
C) {row['choiceC']}
D) {row['choiceD']}
E) {row['choiceE']}

Analisi tecnica: {reasoning}

Basandoti esclusivamente sulla tua analisi sopra, indica la lettera corretta. [/INST]
La risposta corretta è la lettera:"""

        # Otteniamo la predizione via scoring (niente più troncamenti!)
        prediction, confidences = get_logit_scores(model, tokenizer, final_scorer_prompt)

        results.append({
            'custom_id': row.get('custom_id', idx),
            'category': row.get('category', 'N/A'),
            'reasoning': reasoning,
            'prediction': prediction,
            'confidence_score': confidences[prediction],
            'correct_answer': row.get('correct_answer', None),
            'full_confidences': confidences
        })

    return pd.DataFrame(results)

In [ ]:
df_results = run_generate_and_read(validation_data, model, tokenizer)

# Calcolo Accuratezza (se correct_answer è presente)
if 'correct_answer' in df_results.columns and df_results['correct_answer'].notnull().all():
    accuracy = (df_results['prediction'] == df_results['correct_answer']).mean()
    print(f"\n✅ Accuratezza Finale Generate-and-Read: {accuracy:.2%}")

Pipeline G&R:   0%|          | 0/500 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Pipeline G&R: 100%|██████████| 500/500 [2:37:24<00:00, 18.89s/it]


✅ Accuratezza Finale Generate-and-Read: 46.40%


In [ ]:
df_results.head()

,custom_id,category,reasoning,prediction,confidence_score,correct_answer,full_confidences
0,BOOKS__1646,BOOKS,1. Il committente massimizza i propri profitti...,D,0.701660,B,"{'A': 0.004138946533203125, 'B': 0.28344726562..."
1,PAPER__3536,PAPER,1. Il sistema bancario ombra non crea debiti i...,D,0.719238,C,"{'A': 0.05462646484375, 'B': 0.016525268554687..."
2,PAPER__3297,PAPER,1. L'equilibrio competitivo richiede che ogni ...,E,0.503418,C,"{'A': 0.0168304443359375, 'B': 0.273681640625,..."
3,BOOKS__5422,BOOKS,"Per analizzare la domanda, è necessario capire...",C,1.000000,C,"{'A': 2.2649765014648438e-06, 'B': 4.768371582..."
4,PAPER__2502,PAPER,1. ∆CoVaR misura la variazione del valore a ri...,D,0.999512,D,"{'A': 4.947185516357422e-05, 'B': 0.0001465082..."


In [ ]:
import random
import numpy as np
from tqdm import tqdm

def shuffle_options_in_df(df, seed=42):
    """
    Rimescola le opzioni A, B, C, D, E per ogni riga e aggiorna correct_answer.
    """
    random.seed(seed)
    shuffled_rows = []

    for _, row in df.iterrows():
        new_row = row.copy()

        # 1. Mappa originale delle opzioni
        labels = ['A', 'B', 'C', 'D', 'E']
        original_choices = {l: row[f'choice{l}'] for l in labels}
        correct_text = original_choices[row['correct_answer']]

        # 2. Rimescolamento dei testi
        text_list = list(original_choices.values())
        random.shuffle(text_list)

        # 3. Riassegnazione e ricerca della nuova risposta corretta
        for i, label in enumerate(labels):
            new_row[f'choice{label}'] = text_list[i]
            if text_list[i] == correct_text:
                new_row['correct_answer'] = label

        shuffled_rows.append(new_row)

    return pd.DataFrame(shuffled_rows)


In [ ]:
# --- ESECUZIONE DEL TEST DI ROBUSTEZZA ---

# 1. Creiamo il dataset "perturbato"
print("🔄 Generazione del dataset rimescolato...")
df_val_shuffled = shuffle_options_in_df(validation_data)

🔄 Generazione del dataset rimescolato...


In [ ]:
# 2. Lanciamo la pipeline Generate-and-Read sul dataset rimescolato
# Usiamo la funzione run_generate_and_read definita nello step precedente
print("🚀 Esecuzione pipeline G&R su opzioni rimescolate...")
df_results_shuffled = run_generate_and_read(df_val_shuffled, model, tokenizer)

🚀 Esecuzione pipeline G&R su opzioni rimescolate...


Pipeline G&R: 100%|██████████| 500/500 [2:38:06<00:00, 18.97s/it]


In [ ]:
# 3. Analisi comparativa
original_acc = 0.4640
shuffled_acc = (df_results_shuffled['prediction'] == df_results_shuffled['correct_answer']).mean()

In [ ]:
print("\n" + "="*30)
print("📊 ANALISI DI ROBUSTEZZA")
print(f"Accuratezza Originale: {original_acc:.2%}")
print(f"Accuratezza Shuffled:   {shuffled_acc:.2%}")
print(f"Delta Performance:     {shuffled_acc - original_acc:+.2%}")
print("="*30)

if abs(shuffled_acc - original_acc) < 0.03:
    print("✅ Il modello è ROBUSTO: le decisioni sono basate sulla logica, non sulla posizione.")
else:
    print("⚠️ Il modello mostra BIAS POSIZIONALE: le risposte variano in base all'ordine.")


📊 ANALISI DI ROBUSTEZZA
Accuratezza Originale: 46.40%
Accuratezza Shuffled:   49.00%
Delta Performance:     +2.60%
✅ Il modello è ROBUSTO: le decisioni sono basate sulla logica, non sulla posizione.


## Agentic Behavior and Retrieval

## Compare Results

# Spunti di riflessione:


*   Fine-Tuning con QLoRa andando a splittare noi i dati a disposizione un po' come cazz ci pare

* Usare modelli prettamente Italiani
